In [ ]:
import pandas as pd

from aust_covid.inputs import get_ifrs
from emutools.tex import StandardTexDoc
from aust_covid.inputs import get_base_vacc_data
from aust_covid.vaccination import add_derived_data_to_vacc, get_model_vacc_vals_from_data
from aust_covid.model import build_model
from aust_covid.plotting import plot_immune_props
from inputs.constants import PROJECT_PATH, SUPPLEMENT_PATH
from emutools.parameters import load_param_info

from aust_covid.inputs import load_pop_data

In [ ]:
from summer2.functions.time import get_linear_interpolation_function as linear_interp

In [ ]:
vacc_df = get_base_vacc_data()
ext_vacc_df = add_derived_data_to_vacc(vacc_df)
# boost_data = get_model_vacc_vals_from_data(ext_vacc_df, 'prop boosted in preceding')
# primary_data = get_model_vacc_vals_from_data(ext_vacc_df, 'prop primary full in preceding')

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')
param_info = load_param_info()
param_info['value'].update(get_ifrs(app_doc))
parameters = param_info['value'].to_dict()
epi_model = build_model(app_doc, vacc_sens=True)
epoch = epi_model.get_epoch()

In [ ]:
ext_vacc_df['primary full'].dropna().plot()

In [ ]:
ext_vacc_df['prop incremental primary full'].dropna().plot()

In [ ]:
# boost_func = linear_interp(epoch.dti_to_index(ext_vacc_df.index), ext_vacc_df['prop incremental adult booster'])

In [ ]:
# from inputs.constants import AGE_STRATA

In [ ]:
# for comp in epi_model._original_compartment_names:
#     for age_strat in AGE_STRATA[3:]:
#         epi_model.add_transition_flow(
#             'vaccination',
#             boost_func,
#             source=comp.name,
#             dest=comp.name,
#             source_strata={'immunity': 'nonimm', 'agegroup': str(age_strat)},
#             dest_strata={'immunity': 'imm', 'agegroup': str(age_strat)},
#         )

In [ ]:
epi_model.run(parameters=parameters)

In [ ]:
init_pop = epi_model.get_initial_population(parameters=parameters)

In [ ]:
model_pops = load_pop_data(app_doc)
states = epi_model.stratifications['states'].strata
for age in epi_model.stratifications['agegroup'].strata:
    for state in states:
        model_pops.loc[int(age), f'check_{state}'] = init_pop[init_pop.index.str.contains(f'states_{state}') & init_pop.index.str.contains(f'agegroup_{age}X')].sum()

imm_check = {}
for imm in epi_model.stratifications['immunity'].strata:
    imm_check[imm] = init_pop[init_pop.index.str.contains(f'immunity_{imm}')].sum()

print([v / sum(imm_check.values()) for v in imm_check.values()])

In [ ]:
model_pops.plot()

In [ ]:
vacc_df['adult booster'].plot()

In [ ]:
fig = plot_immune_props(epi_model, ext_vacc_df)

In [ ]:
vacc_df['adult booster'] / vacc_df['National - Population 16 and over']

In [ ]:
from plotly import graph_objects as go
fig.add_trace(go.Scatter(x=vacc_df.index, y=vacc_df['adult booster'] / vacc_df['National - Population 16 and over']), row=1, col=2)
fig